## Training Playground
All my training testing through this notebook. 

In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import torch
from nn import MotifCaller, NaiveCaller
from training_data import data_preproc, load_training_data
from utils import get_savepaths, load_model
from Bio import Align
import numpy as np
from sklearn.preprocessing import normalize
from greedy_decoder import GreedyCTCDecoder
from Levenshtein import ratio
from utils import load_model, get_metrics_for_evaluation, sort_transcript
from sklearn.model_selection import train_test_split
from beam_search_decoder import beam_search_ctc
import torch.nn as nn
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:

model_path = r""

dataset_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\synthetic\pickled_datasets\no_spacers_long.pkl"

n_classes = 9
hidden_size = 256

In [5]:
labels_int = np.arange(n_classes).tolist()
labels = [f"{i}" for i in labels_int] # Tokens to be fed into greedy decoder
greedy_decoder = GreedyCTCDecoder(labels = labels)
ctc = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_device(device)

In [6]:
dataset = pd.read_pickle(dataset_path)

In [10]:
X, y = load_training_data(
       dataset_path, column_x='squiggle', column_y='motif_seq', payload=False, sampling_rate=0.2)

X = data_preproc(X, window_size=1000, step_size=300, normalize_values=True)

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

8580
Selected 8580 forward reads


100%|██████████| 1716/1716 [00:03<00:00, 453.32it/s]


In [13]:

if model_path == "":
    model = NaiveCaller(hidden_dim=hidden_size, num_classes=n_classes).to(device)
else:
    model = load_model(
        model_path=model_path, device=device, n_classes=n_classes, hidden_size=hidden_size).to(device)

### Training Loop

In [ ]:
for x, y in zip(X_train, y_train):

    input_sequence = x.to(device)
    target_sequence = torch.tensor(y).to(device)
    input_sequence = input_sequence.view(1, input_sequence.shape[0], input_sequence.shape[-1])
    print(input_sequence.shape)

    model_output = model(input_sequence)
    model_output = model_output.permute(1, 0, 2)  # Assuming log probs are computed in network
    print(model_output.shape)
    model_output = model_output.reshape(
        model_output.shape[0] * model_output.shape[1], n_classes)
    print(model_output.shape)
    

    n_timesteps = model_output.shape[0]
    input_lengths = torch.tensor([n_timesteps])
    label_lengths = torch.tensor([len(target_sequence)])
    
  
    loss = ctc(
        log_probs=model_output, targets=target_sequence, input_lengths=input_lengths, target_lengths=label_lengths)
    print(loss.item())
    break

torch.Size([66, 1000])


RuntimeError: Given groups=1, weight of size [32, 1, 3], expected input[1, 66, 1000] to have 1 channels, but got 66 channels instead